In [ ]:
#!apt-get install dssp --> USE "conda install -c salilab dssp" THIS IS VERSION 3
#!pip install prody --> conda is also better for this if possible

#Only change here was to add the entire filepath to the PDBs when loading them, including using double '/'


import os
import prody as pr
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def distancematrix(AtomMap):
  """Generate a distance matrix from the AtomMap"""    
  coordinates = []

  for pi in AtomMap.select('calpha').getResnums():

    # Check for existance of sidechain, if exists, take center of mass of sidechain, if not, take center of mass of calpha (=calpha)
    if not AtomMap.select('sc and resnum %d' % pi):
      coordinates.append(pr.calcCenter(AtomMap.select('calpha and resnum %d' % pi)))
    else:
      coordinates.append(pr.calcCenter(AtomMap.select('sc and resnum %d' % pi)))

  coordinates = np.array(coordinates)
  distancematrix = pr.buildDistMatrix(coordinates, coordinates)
  return distancematrix

# Library with absolute surface areas: doi:10.1371/journal.pone.0080635
TotAAArea={"ALA": 129.0,
  "ARG": 274.0,
  "ASN": 195.0,
  "ASP": 193.0,
  "CYS": 167.0,
  "GLU": 223.0,
  "GLN": 225.0,
  "GLY": 104.0,
  "HIS": 224.0,
  "ILE": 197.0,
  "LEU": 201.0,
  "LYS": 236.0,
  "MET": 224.0,
  "PHE": 240.0,
  "PRO": 159.0,
  "SER": 155.0,
  "THR": 172.0,
  "TRP": 285.0,
  "TYR": 263.0,
  "VAL": 174.0 }

# Find the distance parameters for user-defined pairs
def findResPair(pdbA, pdbB, resnlist):
    
  # Get entries from PDB
  p = pr.parsePDB(pdbB)
  q = pr.parsePDB(pdbA)

  # Execute DSSP calculation for downloaded PDB files
  pr.execDSSP(os.path.join(os.getcwd(), pdbB + ".pdb.gz"))
  pr.execDSSP(os.path.join(os.getcwd(), pdbA + ".pdb.gz"))
  tp = pr.parseDSSP(os.path.join(os.getcwd(), pdbB + ".dssp"), p)
  tq = pr.parseDSSP(os.path.join(os.getcwd(), pdbA + ".dssp"), q)
  
  # Align sequences
  pm = pr.matchAlign(tp, tq, tarsel = 'protein')[1]
  qm = pr.matchAlign(tp, tq, tarsel = 'protein')[2]

  # Generate distance matrices
  confA = distancematrix(qm)
  confB = distancematrix(pm)
  diff = confA - confB

  # Get the solutions that are non zero and define an outputlist
  solution = np.transpose(np.nonzero(diff))
  
  solutionResidues = []

  #Search for provided residue pairs and append them to the outputlist
  for resni, resnj in resnlist:

    for i, j in solution:

      acc_qm_i = round(100 * (qm.select('calpha').getData('dssp_acc')[i] / TotAAArea[qm.select('calpha').getResnames()[0]]))
      acc_qm_j = round(100 * (qm.select('calpha').getData('dssp_acc')[j] / TotAAArea[qm.select('calpha').getResnames()[0]]))
      acc_pm_i = round(100 * (pm.select('calpha').getData('dssp_acc')[i] / TotAAArea[pm.select('calpha').getResnames()[0]]))
      acc_pm_j = round(100 * (pm.select('calpha').getData('dssp_acc')[j] / TotAAArea[pm.select('calpha').getResnames()[0]]))
      
      if pm.select('calpha').getResnums()[i] == resni and pm.select('calpha').getResnums()[j] == resnj:

        residue1 = "%s%d" % (pm.select('calpha').getResnames()[i], pm.select('calpha').getResnums()[i])
        residue2 = "%s%d" % (qm.select('calpha').getResnames()[j], qm.select('calpha').getResnums()[j])
        
        values = []
        values.append("%s with %s" % (residue1, residue2))
        values.append(diff[i, j])
        values.append(confA[i, j])
        values.append(confB[i, j])
        values.append(acc_pm_i)
        values.append(acc_pm_j)
        values.append(acc_qm_i)
        values.append(acc_qm_j)

        solutionResidues.append(tuple(values))
  
  columns = ("residue pair", "distance change", "distance confA", "distance confB", "surfAccRes1ConfB", "surfAccRes2ConfB", "surfAccRes1ConfA", "surfAccRes2ConfA")
  print("%20s %16s %16s %16s %16s %16s %16s %16s" % columns)
  for values in solutionResidues:
    print("%20s %16.2f %16.2f %16.2f %16.2f %16.2f %16.2f %16.2f" % values)

# Find new pairs based on user-defined parameters
def findResPairs(pdbA, pdbB, distanceChange=7, distanceAbsoluteUpper=78, distanceAbsoluteLower=37, acc_threshold=40):    
  # Get entries from PDB
  # I can replace these variables with my PDB structures
  p = pr.parsePDB("//Users//littlefrank//Downloads//ResiduePairs-main//"+pdbB+ '.pdb')
  q = pr.parsePDB("//Users//littlefrank//Downloads//ResiduePairs-main//"+pdbA+ '.pdb')
  #p = pr.parsePDB(pdbB+ '.pdb')
  #q = pr.parsePDB(pdbA+ '.pdb')
    
  # Execute DSSP calculation for downloaded PDB files
  pr.execDSSP(os.path.join(os.getcwd(), pdbB + '.pdb'))
  pr.execDSSP(os.path.join(os.getcwd(), pdbA + '.pdb'))
  tp = pr.parseDSSP(os.path.join(os.getcwd(), pdbB +'.dssp'), p)
  tq = pr.parseDSSP(os.path.join(os.getcwd(), pdbA +'.dssp'), q)

  # Align sequences
  pm = pr.matchAlign(tp, tq, tarsel='protein')[1]
  qm = pr.matchAlign(tp, tq, tarsel='protein')[2]

  # Generate distance matrices
  confA = distancematrix(qm)
  confB = distancematrix(pm)
  diff = confA - confB
  resid = diff.copy()

  # Apply distance constraints
  # Fill in with my constraints (assume we are using cys 3 and 5)
  resid[confA < distanceAbsoluteLower] = 0
  resid[confA > distanceAbsoluteUpper] = 0 
  resid[confB < distanceAbsoluteLower] = 0 
  resid[confB > distanceAbsoluteUpper] = 0
  resid_abs = resid.copy()
    
  resid[np.abs(diff) < distanceChange] = 0
  resid_dist = resid.copy()
  
  # Get the solutions that are non zero and define an outputlist
  solution = np.transpose(np.nonzero(np.triu(resid)))
  
  solutionResidues = []

  # Generate a PyMol file for easy visualization
  output = open("Output_" + pdbA + ".pml", "w")
  output.write('fetch ' + pdbA + '\n hide everything \n show cartoon,' + pdbA + '\n color green,' + pdbA + '\n')

  for i, j in solution:
    acc_qm_i = round(100 * (qm.select('calpha').getData('dssp_acc')[i] / TotAAArea[qm.select('calpha').getResnames()[0]]))
    acc_qm_j = round(100 * (qm.select('calpha').getData('dssp_acc')[j] / TotAAArea[qm.select('calpha').getResnames()[0]]))
    acc_pm_i = round(100 * (pm.select('calpha').getData('dssp_acc')[i] / TotAAArea[pm.select('calpha').getResnames()[0]]))
    acc_pm_j = round(100 * (pm.select('calpha').getData('dssp_acc')[j] / TotAAArea[pm.select('calpha').getResnames()[0]]))

    if acc_pm_i > acc_threshold and acc_qm_i > acc_threshold and acc_pm_j > acc_threshold and acc_qm_j > acc_threshold:
            
      residue1 = "%s%d" % (pm.select('calpha').getResnames()[i], pm.select('calpha').getResnums()[i])
      residue2 = "%s%d" % (qm.select('calpha').getResnames()[j], qm.select('calpha').getResnums()[j])
      
      values = []
      values.append("%s with %s" % (residue1, residue2))
      values.append(diff[i, j])
      values.append(confA[i, j])
      values.append(confB[i, j])
      values.append(acc_pm_i)
      values.append(acc_pm_j)
      values.append(acc_qm_i)
      values.append(acc_qm_j)

      solutionResidues.append(tuple(values))

      if resid[i,j] < 0:
        print(np.min(resid))
        output.write('create pairs_neg, '+pdbA+' and (resi '+str(pm.select('calpha').getResnums()[i])+' or resi '+str(pm.select('calpha').getResnums()[j])+') and name ca, 0, -1\n')
      if resid[i,j] > 0:
        output.write('create pairs_pos, '+pdbA+' and (resi '+str(pm.select('calpha').getResnums()[i])+' or resi '+str(pm.select('calpha').getResnums()[j])+') and name ca, 0, -1\n')        
    else:
      resid[i,j] = 0
      resid[j,i] = 0


  #Close PyMol file
  output.write('show spheres, pairs_pos'+'\n color blue, pairs_pos'+'\n show spheres, pairs_neg'+'\n color red, pairs_neg'+'\n')
  output.close()

  #print and visualize the solution pairs
  print('%d pairs found!' % len(solutionResidues))

  figure=plt.figure(figsize=(16,6))
  f, axarr = plt.subplots(2,3,sharey='row', sharex='col')
  f.subplots_adjust(wspace=0.05)
  f.subplots_adjust(hspace=0.08)
  plt.rcParams['svg.fonttype'] = 'none'
  plt.rcParams['pdf.fonttype'] = 42
  plt.rcParams['ps.fonttype'] = 42
  plt.rcParams['figure.dpi']= 300
  f.set_figheight(15)
  f.set_figwidth(20)

  axarr[0,0].set_title(pdbA)
  axarr[0,0].imshow(confA,cmap='Blues')
  axarr[0,1].set_title(pdbB)
  axarr[0,1].imshow(confB,cmap='Blues')
  axarr[0,2].set_title('Difference')
  axarr[0,2].imshow(diff, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')
  
  axarr[1,0].set_title('Absolute distance constraints')
  axarr[1,0].imshow(resid_abs, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')
  axarr[1,1].set_title('Distance shift constraints')
  axarr[1,1].imshow(resid_dist, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')
  axarr[1,2].set_title('Picked Pairs')
  axarr[1,2].imshow(resid, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), cmap='RdBu')
  
  print(np.min(resid))
  
  for j in range(2):
    axarr[j,0].set_ylabel('Residue number')
    for i in range(3):    
      axarr[1,i].set_xlabel('Residue number')
      axarr[j,i].set_xticks(np.arange(pm[0].getResnum()-pm[0].getResnum(), pm[-1].getResnum()-pm[0].getResnum(), step=40))
      axarr[j,i].set_yticks(np.arange(pm[0].getResnum()-pm[0].getResnum(), pm[-1].getResnum()-pm[0].getResnum(), step=40))    
      axarr[j,i].set_xticklabels(np.arange(pm[0].getResnum(), pm[-1].getResnum(), step=40))
      axarr[j,i].set_yticklabels(np.arange(pm[0].getResnum(), pm[-1].getResnum(), step=40))
  f.savefig('output'+pdbA+'.pdf', format='pdf')

  columns = ("residue pair", "distance change", "distance confA", "distance confB", "surfAccRes1ConfB", "surfAccRes2ConfB", "surfAccRes1ConfA", "surfAccRes2ConfA")
  print("%20s %16s %16s %16s %16s %16s %16s %16s" % columns)
  for values in solutionResidues:
    print("%20s %16.2f %16.2f %16.2f %16.2f %16.2f %16.2f %16.2f" % values)
  return solutionResidues


In [ ]:
#This section generates control comparisons of all structures within an experiment to ensure they are similar to each other
#Structures must be in the same directory as this program

#Put experiments with their respective structures here in a list (filename without .pdb)
closedMalate = ['MLT1','MLT3','MLT5']
openProtein = ['CON2','CON3','CON4','CON5','CON6']

#Select which experiment to run
substituents = openProtein

for i in range(len(substituents)):
    for j in range(i+1, len(substituents)):
        print("Now comparing: "+substituents[i]+" and "+substituents[j])
        findResPairs(substituents[i],substituents[j], distanceChange=7, acc_threshold=40)
        


In [ ]:
#This section compares the composite control structure (closed) with all structures in other experiments
import pandas as pd
import re 
import pytraj as pt
import numpy as np

def medLargeDist(distance, cutoff):
    test0=np.asarray(distance)
    test = test0[np.logical_not(np.isnan(test0))]
    n=int(len(test)*cutoff)
    idx = np.argpartition(test, -n)[-n:]
    indices = idx[np.argsort((-test)[idx])]
    c=np.median(test[indices])
    return c

def medSmallDist(distance, cutoff):
    test0=np.asarray(distance)
    test = test0[np.logical_not(np.isnan(test0))]
    n=int(len(test)*cutoff)
    idx = np.argpartition(test, n)[:n]
    indices = idx[np.argsort((-test)[idx])]
    c=np.median(test[indices])
    return c


#Control structure
closed='CLOS'

#Put experiments with their respective structures here in a list (filename without .pdb)
closedMalate = ['MLT1','MLT3','MLT5']
openProtein = ['CON2','CON3','CON4','CON5','CON6']

#Select which experiment to run
substituents = openProtein

#commented out so it doesn't waste time while I test out testing
respairs=[]
for exp in substituents:
    print("Now comparing: "+exp+" and "+closed)
    solres=findResPairs(exp, closed, distanceChange=7, acc_threshold=20)
    for respair in solres:
        respairs.append(list(respair))

df = pd.DataFrame(respairs, columns = ['residue_pair', 'delta_distance', 'dist_open', 'dist_closed', 'res1_RSA_closed', 'res2_RSA_closed', 'res1_RSA_open', 'res2_RSA_open'])

df.to_csv('ResiduePairs.csv', index=False)#save data so it does not need to generated again, although it doesn't take too long

#Convert nice readable text of residue pairs to something more easily dealt with later in the program
interestingpairs=[]
for row in df['residue_pair'].unique():
    pair=row.split(' with ')
    pair[0]=re.sub("[^0-9]", "", pair[0])
    pair[1]=re.sub("[^0-9]", "", pair[1])
    interestingpairs.append(pair)

exppath = '/Users/littlefrank/Documents/Biowulf_Data/MATC_setup_and_sim/MLT_D'
controlpath = '/Users/littlefrank/Documents/Biowulf_Data/MATC_setup_and_sim/Control_Protein'

#define experimental and control traces
exps = ['MLT_2','MLT_4','MLT_6']
controls = ['C2','C3','C4','C5','C6']
constart = [2340, 2760, 1380, 1610, 1265] #starting points of control runs where it equillibrates in the open form
#Now I will just load Steps 4-8, quick and dirty method for now while I figure out a workaround to what appears to be memory issues

#give names of trajectory step files
steps = ['step5_1.nc', 'step5_2.nc', 'step5_3.nc', 'step5_4.nc', 'step5_5.nc', 'step5_6.nc', 'step5_7.nc', 'step5_8.nc']
topology = 'step3_input.parm7'

expdists=[] #includes experiment, residue pair, ∆distance, and distance timeseries
trajs=[] #holds the trajectories
for exp in exps:
    directory = os.path.join(exppath, exp)
    trajsteps=[]
    for step in steps:
        trajsteps.append(os.path.join(directory, step))
    trajs.append(pt.iterload(trajsteps, top=os.path.join(directory, topology)))
    
minmaxinfo=[]
for pair in interestingpairs:
    mask = ":"+str(pair[0])+" :"+str(pair[1])
    avgdeldist=0
    for i in range(len(exps)):
        distance = pt.distance(trajs[i], mask)
        distance[distance==0]=np.nan
        deldist = medLargeDist(distance,.05)-medSmallDist(distance,.05) #np.nanmax(distance)-np.nanmin(distance)
        expdists.append((exps[i], pair, deldist, tuple(distance)))
        avgdeldist+=deldist
    avgdeldist=avgdeldist/len(exps)
    minmaxinfo.append((pair, avgdeldist))
print(minmaxinfo)


#now for control runs
controldists=[] #includes experiment, residue pair, ∆distance, and distance timeseries
trajs=[] #holds the trajectories
for i in range(len(controls)):
    directory = os.path.join(controlpath, controls[i])
    trajsteps=[]
    for step in steps:
        trajsteps.append(os.path.join(directory, step))
    traj = pt.iterload(trajsteps, top=os.path.join(directory, topology))
    trajs.append(traj)
print(trajs)
    
minmaxcontrol=[]
for pair in interestingpairs:
    mask = ":"+str(pair[0])+" :"+str(pair[1])
    avgdeldist=0
    for i in range(len(controls)):
        distance = pt.distance(trajs[i], mask)[constart[i]:]
        distance[distance==0]=np.nan
        deldist = medLargeDist(distance,.05)-medSmallDist(distance,.05) #np.nanmax(distance)-np.nanmin(distance)
        controldists.append((controls[i], pair, deldist, tuple(distance)))
        avgdeldist+=deldist
    avgdeldist=avgdeldist/len(controls)
    minmaxcontrol.append((pair, avgdeldist))

compare=[]
for i in range(len(minmaxcontrol)):
    compare.append((minmaxcontrol[i][0], minmaxinfo[i][1]-minmaxcontrol[i][1]))


In [ ]:
from matplotlib import pyplot as plt;
#for plotting timeseries of candidate pairs

smoothing=100

fig = plt.figure(figsize=(12,9))
plt.rcParams["figure.autolayout"] = True
plt.title(label='Distance of a residue pair over time', fontsize=24)
plt.xlabel('Frames', fontsize=16) 
plt.ylabel('Distance (Å)', fontsize=16)

expseries=pd.Series(expdists[9*3+1][3])#use index numbers to select pairs to examine
print(expdists[9*3+1][:3])
#plt.plot(expseries.rolling(window=smoothing).mean());

fig2 = plt.figure(figsize=(19.5,7))
plt.rcParams["figure.autolayout"] = True
#Note that these frame numbers correspond to frames WITH offset
#Do not worry about converting offset frame number to actual frames, this program will keep track for you
plt.title(label='Distance of HSD68-ASN139 over time', fontsize=40)
plt.xlabel('Time (ns)', fontsize=30) 
plt.ylabel('Distance (Å)', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)


expseries=pd.Series(expdists[16*3][3])
conseries=pd.Series(controldists[16*5][3])
xvals=list(np.linspace(constart[0]/10, (constart[0]+len(conseries)-1)/10, len(conseries)))
allx=list(np.linspace(0,(len(expseries)-1)/10,len(expseries)))
print(expdists[16*3][:3],controldists[16*5][:3])
plt.plot(allx,expseries.rolling(window=10).mean(), label='Malate unbinding');
plt.plot(xvals,conseries.rolling(window=10).mean(), label='Open form');
print(df['residue_pair'].value_counts())


In [ ]:
#Test the pairs by comparing unbinding events with open form
#Also while I'm at it, remove residues >283 and <4, note this isn't exactly the residue number since the crystal structure is missing some residues at the ends

def Sort(sub_li):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[1]))    
  

goodpairs=[]
allres=[]
cutoff=.5
print(len(compare))
for i in range(len(compare)):
    if int(compare[i][0][0])<284 and int(compare[i][0][1])<284 and int(compare[i][0][0])>2 and int(compare[i][0][1])>2:
        allres.append(int(compare[i][0][0]))
        allres.append(int(compare[i][0][1]))
    if compare[i][1]>cutoff and int(compare[i][0][0])<284 and int(compare[i][0][1])<284 and int(compare[i][0][0])>2 and int(compare[i][0][1])>2:
        goodpairs.append(compare[i])
        

print(Sort(goodpairs)[::-1], len(goodpairs))
